In [1]:
import os, sys, torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:
# model = "mindeye1_40"
model = "brain_diffuser_750_trials"
all_images = torch.load("evals/all_images.pt")
all_recons = torch.load(f"evals/{model}/subj01_{model}_all_recons.pt")
# all_recons = torch.load("/home/naxos2-raid25/kneel027/home/kneel027/Second-Sight/data/preprocessed_data/subject7/mindeye_40_recons.pt")

imsize = 150
if all_images.shape[-1] != imsize:
    all_images = transforms.Resize((imsize,imsize))(all_images).float()
if all_recons.shape[-1] != imsize:
    all_recons = transforms.Resize((imsize,imsize))(all_recons).float()

num_images = all_recons.shape[0]
num_rows = (2 * num_images + 9) // 10

# Interleave tensors
merged = torch.stack([val for pair in zip(all_images, all_recons) for val in pair], dim=0)

# Calculate grid size
grid = torch.zeros((num_rows * 10, 3, all_recons.shape[-1], all_recons.shape[-1]))

# Populate the grid
grid[:2*num_images] = merged
grid_images = [transforms.functional.to_pil_image(grid[i]) for i in range(num_rows * 10)]

# Create the grid image
grid_image = Image.new('RGB', (all_recons.shape[-1]*10, all_recons.shape[-1] * num_rows))  # 10 images wide

# Paste images into the grid
for i, img in enumerate(grid_images):
    grid_image.paste(img, (all_recons.shape[-1] * (i % 10), all_recons.shape[-1] * (i // 10)))
os.makedirs(f"evals/{model}", exist_ok=True)
grid_image.save(f"evals/{model}/{model}_1000recons.png")

/export/raid1/home/kneel027/miniconda3/envs/mindeyev2/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
